### Projekt 01
**Wybrany model : Las losowy w wariancie Extra Trees.**

Problem badawczy : Wykorzystaną bazą do klasyfikacji będzie zestaw - 'Drum Kit Sound Samples':
*https://www.kaggle.com/datasets/anubhavchhabra/drum-kit-sound-samples*
Klasyfikować, więc będziemy dźwięki perkusyjne w celu rozróżnienia bębnów, perkusji czy werbli, a dokładnie zgodnie z angielską nomenklaturą bazy danych : kick, snare, toms i overheads. W bazie znajduje się dokłądnie 160 sampli audio, gdzie każdy z rodzajów to 40 plików .wav. Nagrania zostały pozyskane drogą nagrań 'live' lub są to dźwięki 'symulowane' techniką komputerową. Bazę można wykorzystać również do zadań klasteryzacyjnych.

Celem i obiktem klasyfikacji jest więc model, który nauczy się rozpoznawać określone warianty dźwięków perkusyjnych, który może być wykorzystany np. w reprodukcji muzyki.

Tak jak i w opisie wyżej, baza zawiera, więc 4 klasy po 40 obiektów, co daje w sumie 'database' składający się ze 160 elementów. Format pliku to .wav, a częstotliwość próbkowania to : 

Wybrane modele do klasyfikacji to :
- Las Losowy (w opcji Extra Trees) [wykonanie - Jakub Gucik]
- Wektory Nośne
- Algorytm K-sąsiadów
- Gaussian Naive Bayes

Każdy z członków zespołu zajmie się jednym z modeli w początkowej fazie, a jeżeli wystarczy czasu każdy z członków przeanalizuje jeszcze jeden dodatkowy model z listy.

Ten Jupyter Notebook jest poświęcony modelowi Lasu Losowego.

### Preprocessing

Preprocessing zostanie przeprowadzony standardowo, zgodnie z tym co pojawiało się na zajęciach, a więc odpowiednie wczytanie i label'owanie nagrań, a także ustandaryzowanie danych za pomocą Standard Scaler'a.

Poniżej import bibliotek :

In [128]:
# Libraries import :

import scipy.stats
import os
import librosa
import pickle
import optuna
import pandas

import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, accuracy_score, recall_score, f1_score, make_scorer, confusion_matrix, log_loss
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, GroupKFold
from pathlib import Path
from sklearn.model_selection import train_test_split

Przygotowanie dostępu do folderów z odpowiednimi klasami dla lepszej organizacji i listy do zautomatyzowania wczytywania nagrań :

In [17]:
# Creation of directories and lists :

repoDir = Path.cwd()
archiveDir = os.path.join(repoDir, "archive")
kickDir = os.path.join(archiveDir, "kick")
overheadsDir = os.path.join(archiveDir, "overheads")
snareDir = os.path.join(archiveDir, "snare")
tomsDir = os.path.join(archiveDir, "toms")

listDir = [kickDir, overheadsDir, snareDir, tomsDir]

Poniżej przygotowanie danych do analizy. W formie Dataframe'u - dla wygody i ułatwienia posługiwania się danymi. Niżej check, czy wszystkie nagrania są w tej samej częstotliwości próbkowania :

In [39]:
# Creation of database :

records = []
wavNames = []
labels = np.append(np.zeros(40), [np.ones(40), np.ones(40)*2, np.ones(40)*3])
for list in listDir:
    os.chdir(repoDir)    
    currentList = os.listdir(list)
    os.chdir(list)
    for recording in currentList:
        records.append(librosa.load(recording))
        wavNames.append(recording)

database = pandas.DataFrame(data=records)
database.columns = ["record_data", "sampling_frequency"]
database["audio_name"] = wavNames
database["labels"] = labels
display(database)
os.chdir(repoDir)

# Sampling frequency check :

for frequency in database["sampling_frequency"]:
    if frequency == 22050:
        pass
    else:
        print("Freqency doesn't match 22050 Hz.")

,record_data,sampling_frequency,audio_name,labels
0,"[-0.0023142865, 0.04351061, 0.22839355, 0.1913...",22050,Bass Sample 1.wav,0.0
1,"[-0.0062833373, 0.03805868, 0.16618551, 0.1538...",22050,Bass Sample 10.wav,0.0
2,"[-0.010531046, 0.07673455, 0.17126107, 0.15169...",22050,Bass Sample 11.wav,0.0
3,"[-0.01024509, 0.08281813, 0.23883936, 0.141605...",22050,Bass Sample 12.wav,0.0
4,"[0.0012859756, 0.0013315802, 0.0022426718, 0.0...",22050,Bass Sample 13.wav,0.0
...,...,...,...,...
155,"[-1.2331269e-05, 2.0280546e-05, -2.9059745e-05...",22050,Tom Sample 5.wav,3.0
156,"[-1.5193141e-05, 1.25373435e-05, -6.9293824e-0...",22050,Tom Sample 6.wav,3.0
157,"[-3.7897775e-05, 3.575749e-05, -2.7921751e-05,...",22050,Tom Sample 7.wav,3.0
158,"[1.4454002e-05, -1.2042713e-05, 6.8729264e-06,...",22050,Tom Sample 8.wav,3.0


Kolejno trzeba wyekstrachować cechy. Użyte zostanie 13 cech MFCC, a dodatkowo wykorzystane zostanie MFCC_delta oraz MFCC_delta_delta. Wszystkie dane zostaną przedstawione w DataFrame'ie :

In [41]:
# Extracting MFCC coefficents

mfccs = []
mfccs_delta = []
mfccs_deltasq = []
for data in database["record_data"]:
    mfcc = librosa.feature.mfcc(y=data, sr=22050, n_mfcc=13)
    mfccs.append(mfcc)
    mfccs_delta.append(librosa.feature.delta(mfcc))
    mfccs_deltasq.append(librosa.feature.delta(mfcc, order=2))

database["mfcc"] = mfccs
database["mfcc_delta"] = mfccs_delta
database["mfcc_delta_delta"] = mfccs_deltasq

display(database)

,record_data,sampling_frequency,audio_name,labels,mfcc,mfcc_delta,mfcc_delta_delta
0,"[-0.0023142865, 0.04351061, 0.22839355, 0.1913...",22050,Bass Sample 1.wav,0.0,"[[-93.479805, -114.144936, -227.11826, -291.86...","[[-39.00709, -39.00709, -39.00709, -39.00709, ...","[[9.838433, 9.838433, 9.838433, 9.838433, 9.83..."
1,"[-0.0062833373, 0.03805868, 0.16618551, 0.1538...",22050,Bass Sample 10.wav,0.0,"[[-56.804806, -82.62502, -188.9184, -255.2661,...","[[-44.079357, -44.079357, -44.079357, -44.0793...","[[9.730593, 9.730593, 9.730593, 9.730593, 9.73..."
2,"[-0.010531046, 0.07673455, 0.17126107, 0.15169...",22050,Bass Sample 11.wav,0.0,"[[-59.668728, -88.136024, -202.50055, -271.192...","[[-46.809498, -46.809498, -46.809498, -46.8094...","[[10.21025, 10.21025, 10.21025, 10.21025, 10.2..."
3,"[-0.01024509, 0.08281813, 0.23883936, 0.141605...",22050,Bass Sample 12.wav,0.0,"[[-96.24065, -120.71324, -238.46904, -298.0796...","[[-40.509735, -40.509735, -40.509735, -40.5097...","[[9.705526, 9.705526, 9.705526, 9.705526, 9.70..."
4,"[0.0012859756, 0.0013315802, 0.0022426718, 0.0...",22050,Bass Sample 13.wav,0.0,"[[-74.85452, -86.7107, -195.75198, -266.0516, ...","[[-42.032085, -42.032085, -42.032085, -42.0320...","[[9.802632, 9.802632, 9.802632, 9.802632, 9.80..."
...,...,...,...,...,...,...,...
155,"[-1.2331269e-05, 2.0280546e-05, -2.9059745e-05...",22050,Tom Sample 5.wav,3.0,"[[-114.22107, -114.82725, -159.41573, -184.753...","[[-21.870008, -21.870008, -21.870008, -21.8700...","[[1.4881332, 1.4881332, 1.4881332, 1.4881332, ..."
156,"[-1.5193141e-05, 1.25373435e-05, -6.9293824e-0...",22050,Tom Sample 6.wav,3.0,"[[-117.78517, -117.24555, -161.8991, -185.3812...","[[-19.858833, -19.858833, -19.858833, -19.8588...","[[1.8323971, 1.8323971, 1.8323971, 1.8323971, ..."
157,"[-3.7897775e-05, 3.575749e-05, -2.7921751e-05,...",22050,Tom Sample 7.wav,3.0,"[[-96.04339, -87.022575, -120.32267, -136.3575...","[[-15.441239, -15.441239, -15.441239, -15.4412...","[[0.24270107, 0.24270107, 0.24270107, 0.242701..."
158,"[1.4454002e-05, -1.2042713e-05, 6.8729264e-06,...",22050,Tom Sample 8.wav,3.0,"[[-60.1574, -62.694893, -126.64831, -157.17746...","[[-23.646395, -23.646395, -23.646395, -23.6463...","[[2.9433775, 2.9433775, 2.9433775, 2.9433775, ..."


Oczywiście, ciężko będzie użyć MFCC, które jest niezgodne co do długości z innymi. Jednak nasze nagrania mają z góry przygotowane nagrania tak, że długości wszystkich MFCC wynoszą : 1131 elementów. Jednak trzeba to sprawdzić kodem poniżej :

In [67]:
mfcc_len = 1131

for item in database["mfcc"]:
    if np.size(item) != mfcc_len:
        print("Lenght of arrays doesn't match !")

for item in database["mfcc_delta"]:
    if np.size(item) != mfcc_len:
        print("Lenght of arrays doesn't match !")

for item in database["mfcc_delta_delta"]:
    if np.size(item) != mfcc_len:
        print("Lenght of arrays doesn't match !")

In [127]:
std_mean = []
for mfcc in database["mfcc"]:
    temp = []
    for coeff in mfcc:      
        np.std()
        np.mean()

TypeError: _vhstack_dispatcher() takes 1 positional argument but 2 were given

In [129]:
X_train, X_test, y_train, y_test = train_test_split(database["mfcc"], database["labels"], test_size=0.2, random_state=42, stratify=database["labels"])

This will be the main file of project.